In [ ]:
from urllib.request import urlopen  
from tqdm import tqdm
from lxml import html
from datetime import datetime
from collections import Counter
import json
import requests
import re
# from screeninfo import get_monitors
import pandas as pd
import random
import os
import ast
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# US

In [80]:
#replace as needed
folder_name = "Supplementary Materials"
followers_path = os.path.join(folder_name, "followers_US.txt")
# followers_path = os.path.join(folder_name, "new republican scrape.txt")
completed_path = os.path.join(folder_name, "completed_accounts")

followers_US = []
# with open(r'''X\followers_US.txt''', 'r') as f: # PC
with open(followers_path,'r') as f: #List of accounts from raw followers list
    for line in f:
        if "@" in line:
            follower = line.strip()[1:]
            if follower not in followers_US:
                # print(follower)
                followers_US.append(follower)
f.close()

# with open(completed_path, 'r') as f: #Get list of accounts that are completed
#     for line in f:
#         if line.startswith('total'):
#             completed_list = ast.literal_eval(line[8:])

#Filter out the ones that are completed
# followers_US_set = set(followers_US)
# followers_US = [f for f in followers_US_set if f not in completed_list]

# followers_US = followers_US[32:]

print(followers_US)

followers_US = ["kylegriffin1"]

print(f"Number of followers to process: {len(followers_US)}")

[]
Number of followers to process: 1


In [81]:
tweets_dict_US = {}

# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

In [82]:
# driver = webdriver.Chrome()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('https://twitter.com/i/flow/login')
wait = WebDriverWait(driver, 10)


In [83]:
your_email = "blaketrn@udel.edu"
your_username = "HaHoangNha55719"
your_password = "Idontevenknow1!"

### Log in

# Username
username = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete=username]'))
)
username.send_keys(your_email)
time.sleep(3)
login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[role=button].r-13qz1uu'))
)
login_button.click()
time.sleep(3)

try: 
    # Password
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[type=password]'))
    )
    password.send_keys(your_password)
    time.sleep(3)
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid*=Login_Button]'))
    )
    login_button.click()
    time.sleep(3)
    
except: # if we get an error
    username = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextTextInput]"))
    )
    username.send_keys(your_username)
    time.sleep(3)
    
    next_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextNextButton]"))
    )
    next_button.click()
    time.sleep(3)
    
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[name*=password]"))
    )
    password.send_keys(your_password)
    time.sleep(3)
    
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=LoginForm_Login_Button]"))
    )
    login_button.click()
    time.sleep(3)
    

In [ ]:
def generate_date_ranges(start_date, end_date):
    """
    Generate monthly date ranges between start_date and end_date
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        
    Returns:
        list of tuples: Each tuple contains (since_date, until_date) strings
    """
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    
    date_ranges = []
    current = start
    
    while current < end:
        next_month = current + relativedelta(months=1)
        if next_month > end:
            next_month = end
            
        date_ranges.append((
            current.strftime('%Y-%m-%d'),
            next_month.strftime('%Y-%m-%d')
        ))
        current = next_month
        
    return date_ranges

In [ ]:
count = 1

keyword1 = "(Gaza* OR Israel* OR (West Bank) OR (Tel Aviv) OR (Tel Aviv-Yafo) OR Bedouin* OR Palestin* OR Middle OR East* OR Middle OR Eastern OR Jew* OR rabbi OR Muslim* OR Islam* OR Jihad OR Antisemiti* OR Zionis* OR IDF)"
keyword2 = "(IOF OR Hamas OR Massacre OR Genocide OR Ceasefire OR Terroris* OR Netanyahu OR Histadrut OR Haniyeh OR Yahya OR Sinwar OR Fatah OR Mohammed OR Deif OR Rafah OR Khan OR Younis OR Ben-Gvir OR Abbas OR Gallant)"
hashtags = "(# OR #IsraelPalestineWar OR #IsraelGazaWar OR #Gazabombing OR #CeasefireNOW OR #FreePalestine OR #StrikeForGaza OR #Gazagenocide)"


# AFTER
# until = "2024-10-07"
# since = "2023-10-07"

#BEFORE
# until = "2023-10-06"
# since = "2022-10-07"

# Configure your date range here
start_date = "2023-10-07"  # Starting date
end_date = "2024-10-07"    # Ending date

date_ranges = generate_date_ranges(start_date, end_date)

unprocessed = []

for profile in tqdm(followers_US):
    if count%5 == 0:
        print("5 Accounts passed, sleeping for 5 minutes.")
        time.sleep(300)
    
    if profile not in tweets_dict_US.keys():
        print(f"Processing: {profile}")

        tweets_dict_US[profile] = []

        for since, until in date_ranges:
            print(f"\nScraping period: {since} to {until}")

        ### Begin scrape
        time.sleep(3)

        # Search
        search_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search and explore"]'))
        )
        search_button.click()
        time.sleep(3)

        search_bar = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search"]'))
        )
        search_bar.click()
        time.sleep(3)

        for i in range(2): 
            search_query = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search query"]')))

            # Advanced search query
            if i == 0:
                search_query.send_keys(f'{keyword1} {hashtags} (from:@{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')
            if i == 1:
                search_query.send_keys([Keys.BACKSPACE]*1000)
                search_query.send_keys(f'{keyword2} {hashtags} (from:@{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')

            search_query.send_keys(Keys.RETURN)
            time.sleep(6)
            
            # Scroll until you cannot scroll anymore
            reached_page_end = False
            last_height = driver.execute_script("return document.body.scrollHeight")

            retry_xpath = "//button[@role='button' and @type='button']//span[text()='Retry']"
            # random.randrange(10, 15)
            if (is_element_present(driver, retry_xpath)):
                try:
                    print(f'{profile} - {count} - Retry button appeared.')
                    unprocessed.append(profile)
                    for i in range(5):
                        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, retry_xpath)))
                        search_button.click()
                    time.sleep(100) 
                except:
                    print(f"{profile} - Failed to click Retry")

            show_more_xpath = "//div[@data-testid='tweet-text-show-more-link']"
            return_button_xpath = ""
            while not reached_page_end:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")   
                time.sleep(6)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if last_height == new_height:
                    reached_page_end = True
                else:
                    last_height = new_height
                # Check for Show more Button
                if is_element_present(driver, show_more_xpath):
                    try:
                        # Wait for element and scroll it into view
                        show_more_button = wait.until(
                            EC.presence_of_element_located((By.XPATH, show_more_xpath))
                        )
                        
                        # Scroll element into middle of viewport
                        driver.execute_script(
                            "arguments[0].scrollIntoView({block: 'center', inline: 'center'});", 
                            show_more_button
                        )
                        time.sleep(1)  # Let scroll complete
                    
                        show_more_button = wait.until(
                            EC.element_to_be_clickable((By.XPATH, show_more_xpath))
                        )
                        show_more_button.click()
                    
                        # Wait for content to load
                        time.sleep(3)
                        
                        # Get tweet content
                        search_results = driver.find_element(
                            By.CSS_SELECTOR, 
                            'div[data-testid="cellInnerDiv"]'
                        )
                        tweets_listed = search_results.get_attribute('innerHTML')
                        tweets_dict_US[profile].append(tweets_listed)
                        
                        # Click back button
                        back_button = wait.until(
                            EC.element_to_be_clickable(
                                (By.CSS_SELECTOR, 'button[data-testid="app-bar-back"]')
                            )
                        )
                        back_button.click()
                        time.sleep(2)  # Wait for navigation
                        
                    except Exception as e:
                        print(f"Error clicking 'Show more' button: {str(e)}")
                        continue
                try:
                    # Select the home timeline div
                    search_results = driver.find_element(By.CSS_SELECTOR, '[aria-label="Timeline: Search timeline"]')
                    # Just get all the HTML, will parse later
                    tweets_listed = search_results.get_attribute('innerHTML')
                    time.sleep(3)

                    # Append to dict
                    tweets_dict_US[profile].append(tweets_listed)

                except:
                    pass

    count+=1

print("Unprocessed accounts for this run:")
for profile in unprocessed:
    print(f"@{profile}\n")


NameError: name 'generate_date_ranges' is not defined

In [85]:
# driver.quit()

In [86]:
tweets_dict_US.keys()

dict_keys(['kylegriffin1'])

In [87]:
tweets_dict_US_cleaned = {}
    
for key, value in tweets_dict_US.items():
    if key not in tweets_dict_US_cleaned.keys():
        tweets_dict_US_cleaned[key] = []
        for v in value:
            soup = BeautifulSoup(v, 'html.parser')
            tweets = soup.find_all(attrs={"data-testid": "tweetText"})
            for tweet in tweets:
                tweet_text = tweet.get_text(strip=True)
                tweets_dict_US_cleaned[key].append(tweet_text)

tweets_dict_US_cleaned.keys()

dict_keys(['kylegriffin1'])

In [88]:
raw_data_folder = os.path.join("Raw Data", "After")
raw_json = os.path.join(raw_data_folder, f"US_tweets_{until}_{since}.json")
# cleaned_json = os.path.join(raw_data_folder, f"US_tweets_cleaned_{until}_{since}.json")

In [89]:
with open(raw_json, 'w') as file:
    json.dump(tweets_dict_US, file)

In [90]:
# with open(cleaned_json, 'w') as file:
#     json.dump(tweets_dict_US_cleaned, file)